**This problem set is due Wednesday, October 15, 2025 at 11:59 pm. Please plan ahead and submit your work on time.**

# Problem Set 05: Adversarial Games

In this problem set, you will explore minimax, alpha-beta pruning, and expectimax.

0. [Credit for Contributors (required)](#contributors)

1. [Adversarial Games (100 points)](#problem2)
    1. [MiniMax (50 points)](#part1)
    2. [MiniMax with Alpha-Beta pruning (25 points)](#part2)
    3. [Expectimax (25 points)](#part3)
2. [Time Spent on Pset (5 points)](#part4)

    
**100 points + 5 bonus** total for Problem Set 5

## Imports and Utilities

In [15]:
from __future__ import division
import random
from utils import *

%load_ext autoreload
%autoreload 2
from principles_of_autonomy.grader import Grader
from principles_of_autonomy.notebook_tests.pset_5 import TestPSet5

class Error(Exception):
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <a name="contributors"></a> 0. Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat expectimax problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

No other contributors

## <a name="problem2"></a> 1. Adversarial Games: The Cut-Throat Game of Tic-Tac-Toe

Though the game of tic-tac-toe is very simple, it serves as a good example for MiniMax search, alpha-beta pruning, and Expectimax search because it is fully solvable. In this problem set, you will implement MiniMax search with and without alpha-beta pruning, and compare performance. You will then simulate a full game to prove that an optimal strategy in tic-tac-toe can only guarantee a tie (against an optimal adversary). Then you will implement Expectimax search to play against a suboptimal adversary.

We have defined for you a `game_state` class, which works for general games, and a `tic_tac_toe_board` class, both defined in the file `utils.py`. You may want to look in this file now to see the specifics of these classes.

A `tic_tac_toe_board` object is defined as follows:

```
tic_tac_toe_board(symbols)
```

The `symbols` argument is a list of 9 elements which may be one of `'x'`, `'o'`, or `' '`, corresponding to the symbols on the board supplied row by row.

You will not work directly with a `tic_tac_toe_board`. Instead, a `game_state` object will track the board state of any two player game object supplied to it. It will also provide information on the current player, and the state of the game in the last move. An instance is defined as:

```
game_state(board,parent=None)
```

The `board` argument is any game board (e.g., a tic-tac-toe board), and the `parent` argument refers to the previous game state. If `None` is supplied as the parent, the game is assumed to have started from this state.

To show the current board, you may use `print(game_state)` as below.

In [16]:
# Define a game state for tic-tac-toe
example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ',' ']))
# View the state of the board
print(example_state)

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  



By default, a `game_state` is initialized assuming player 1, who plays x's and attempts to maximize score, is playing first. Player 2 plays o's and attempts to minimize the score. 

We attribute a score of +1 to any game state where x (player 1) wins, -1 to any game state where o (player 2) wins, and 0 to any tied or incomplete game state. You may see the score using `game_state.score()`, and check whether a game is complete using `game_state.terminal_check()`.

In [17]:
# Run this cell to see some example games, who has won, and whether the game is complete.
example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ',' ']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: Game incomplete.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ','x']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: x wins.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o','o','o']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: o wins.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o','x','o']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: Game tie.\n'.format(example_state.score()))

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

Game complete? False
Score of this game is 0: Game incomplete.

x | o | x
--+---+--
x | x | o
--+---+--
o |   | x

Game complete? True
Score of this game is 1: x wins.

x | o | x
--+---+--
x | x | o
--+---+--
o | o | o

Game complete? True
Score of this game is -1: o wins.

x | o | x
--+---+--
x | x | o
--+---+--
o | x | o

Game complete? True
Score of this game is 0: Game tie.



The player who needs to play next in any given `game_state` is given through the command `game_state.player`. The function `game_state.successors()` creates a list of `game_state` objects that could be reachable within a move of the current player. The parent of the successors is automatically allocated.

In [18]:
# Run this cell to see current players and successor states
example_state = game_state(tic_tac_toe_board(['x','o','x','x',' ','o','o',' ',' ']))
print('Example state:')
print(example_state)
print('Next player is {} and the possible successor states are:\n'.format(example_state.player))
successor_states = example_state.successors()
for s in successor_states:
    print(s)
    
print('-----------------------------------------\n')
example_state = successor_states[0]
print('Example state:')
print(example_state)
print('Next player is {} and the possible successor states are:\n'.format(example_state.player))
successor_states = example_state.successors()
for s in successor_states:
    print(s)

Example state:
x | o | x
--+---+--
x |   | o
--+---+--
o |   |  

Next player is 1 and the possible successor states are:

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

x | o | x
--+---+--
x |   | o
--+---+--
o | x |  

x | o | x
--+---+--
x |   | o
--+---+--
o |   | x

-----------------------------------------

Example state:
x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

Next player is 2 and the possible successor states are:

x | o | x
--+---+--
x | x | o
--+---+--
o | o |  

x | o | x
--+---+--
x | x | o
--+---+--
o |   | o



### 1A. <a name="part1"/> </a>Implement MiniMax Search (50 points)

We will use MiniMax to produce a game where both players are following their optimal strategy. You should be able to convince yourself that in this case, the outcome of the entire game can be defined at the start. 

You will define the following functions:

```python
def maximize_score(state: game_state) -> (max_score, states_explored, optimal_play)
```
```python
def minimize_score(state: game_state) -> (min_score, states_explored, optimal_play)
```

Both functions take a `game_state` object as input. It is assumed that when `maximize_score` is run, the next player to play is player 1, while `minimize_score` is run when the next player is player 2.  

The output of `maximize_score` will be a tuple `(max_score, states_explored, optimal_play)`, where: 

- `max_score` is the maximum score that can be achieved starting from the provided state if both players follow an optimum strategy. 
- `states_explored` is the total number of game states that have been considered to perform this computation (including the current state). If your search reaches the same state multiple times, count it multiple times.
- `optimal_play` is a list of `game_state` objects following the optimal game `[state, state2, state3, ..., final_state]` from the current state (leftmost element of list) to its conclusion (rightmost element).

Similarly, the output of `minimize_score` will be a tuple `(min_score, states_explored, optimal_play)`, where:

- `min_score` is the minimum score that can be achieved starting from the provided state if both players follow an optimum strategy. 
- `states_explored` and `optimal_play` are defined as above.

**In the case where multiple moves give a tie for the optimal play, the `optimal_play` list should follow the first successor in the list generated by `state.successors()`.**

We provide you with the base case function `score(state)` (or `value(state)` in lecture). Given a game state, this function runs a terminal check and if the game state is NOT terminal, it calls the minimizer or maximier node (as determined by `state.player`).

<div class="alert alert-info">
Implement functions `maximize_score(state)` and `minimize_score(state)` below.
</div>

In [23]:
# Write your code for MiniMax search here.
class Error(Exception):
    pass

# This is the base case function.
def score(state):
    # Initialize states_explored as 1 (this state) and optimal_play (this state).
    states_explored = 1
    optimal_play = [state]
    
    if state.terminal_check():
        terminal_score = state.score()
        return (terminal_score, states_explored, optimal_play)
    elif state.player == 1:
        return maximize_score(state)
    elif state.player == 2:
        return minimize_score(state)
    
def maximize_score(state):
    """Return (max_score, states_explored, optimal_play) for player 1."""
    if state.terminal_check():
        return state.score(), 1, [state]

    if state.player != 1:
        raise ValueError("maximize_score called on non-player-1 state")

    max_score = -float('inf')
    total_states_explored = 1  # count current state
    best_play = [state]

    # Loop through all successors
    for successor in state.successors():
        succ_score, succ_explored, succ_play = minimize_score(successor)
        total_states_explored += succ_explored

        # Choose the highest scoring successor
        if succ_score > max_score:
            max_score = succ_score
            best_play = [state] + succ_play  # prepend current state

    return max_score, total_states_explored, best_play


def minimize_score(state):
    """Return (min_score, states_explored, optimal_play) for player 2."""
    if state.terminal_check():
        return state.score(), 1, [state]

    if state.player != 2:
        raise ValueError("minimize_score called on non-player-2 state")

    min_score = float('inf')
    total_states_explored = 1
    best_play = [state]

    for successor in state.successors():
        succ_score, succ_explored, succ_play = maximize_score(successor)
        total_states_explored += succ_explored

        # Choose the lowest scoring successor
        if succ_score < min_score:
            min_score = succ_score
            best_play = [state] + succ_play

    return min_score, total_states_explored, best_play

In [24]:
"""Test your MiniMax search code here."""
Grader.run_single_test_inline(TestPSet5, "test_1_minimax", locals())

Test passed!!

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


### 1B. <a name="part2"/> </a>Implement Alpha-Beta Pruning (25 points)

Now you will implement alpha-beta pruning in your MiniMax functions and demonstrate that the same result is achieved with fewer game states needing to be searched. 

You will define the following functions:

```python
def maximize_score_alpha_beta(state,alpha,beta)
def minimize_score_alpha_beta(state,alpha,beta)
```

These functions will look very similar to your implementations of `maximize_score` and `minimize_score` (in fact, a good place to start is by copying and pasting your code from above), except they will take the alpha and beta parameters defined by alpha-beta pruning. The output format should be the same as the corresponding functions without pruning. Once again, we provide you with the base case function `score_alpha_beta`.

<div class="alert alert-info">
Implement functions `maximize_score_alpha_beta(state,alpha,beta)` and `minimize_score_alpha_beta(state,alpha,beta)` below.
</div>

In [26]:
# Write your code for MiniMax search with alpha-beta pruning here.

# This is the base case function.
def score_alpha_beta(state,alpha,beta):
    # Initialize states_explored as 1 (this state) and optimal_play (this state).
    states_explored = 1
    optimal_play = [state]
    
    if state.terminal_check():
        terminal_score = state.score()
        return (terminal_score, states_explored, optimal_play)
    elif state.player == 1:
        return maximize_score_alpha_beta(state,alpha,beta)
    elif state.player == 2:
        return minimize_score_alpha_beta(state,alpha,beta)

def maximize_score_alpha_beta(state, alpha, beta):
    """Maximizer node (Player 1) with alpha-beta pruning."""
    if state.player != 1:
        raise Error

    # Base case: terminal
    if state.terminal_check():
        return state.score(), 1, [state]

    max_score = -float('inf')
    total_states_explored = 1
    best_play = [state]

    # Explore successors
    for successor in state.successors():
        succ_score, succ_explored, succ_play = minimize_score_alpha_beta(successor, alpha, beta)
        total_states_explored += succ_explored

        # Update best score and path
        if succ_score > max_score:
            max_score = succ_score
            best_play = [state] + succ_play

        # Update alpha (best score so far for maximizer)
        alpha = max(alpha, max_score)

        # Alpha-Beta pruning: if alpha >= beta, stop searching
        if alpha >= beta:
            break

    return max_score, total_states_explored, best_play


def minimize_score_alpha_beta(state, alpha, beta):
    """Minimizer node (Player 2) with alpha-beta pruning."""
    if state.player != 2:
        raise Error

    # Base case: terminal
    if state.terminal_check():
        return state.score(), 1, [state]

    min_score = float('inf')
    total_states_explored = 1
    best_play = [state]

    for successor in state.successors():
        succ_score, succ_explored, succ_play = maximize_score_alpha_beta(successor, alpha, beta)
        total_states_explored += succ_explored

        if succ_score < min_score:
            min_score = succ_score
            best_play = [state] + succ_play

        # Update beta (best score so far for minimizer)
        beta = min(beta, min_score)

        # Alpha-Beta pruning
        if alpha >= beta:
            break

    return min_score, total_states_explored, best_play


In [27]:
"""Test your alpha beta search code here."""
Grader.run_single_test_inline(TestPSet5, "test_2_alpha_beta", locals())

Test passed!!

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Simulate a Full Game w/ Optimal Agent

You're now prepared to run a full game of tic-tac-toe from an initially blank state. If it was possible to always guarantee a win against a perfect opponent, you would see x's win the game. Unfortunately, this isn't possible! Let's see this by running your code both with and without alpha-beta pruning.

In [28]:
# Run this cell to see an optimum game of tic-tac-toe. Note the performance WITHOUT alpha-beta pruning.
# This may take a few seconds. 
blank_game = game_state(tic_tac_toe_board([' ',' ',' ',' ',' ',' ',' ',' ',' ']))
(max_score, states_explored, optimal_play) = score(blank_game)
print('Max score = {}'.format(max_score))
print('States explored = {}'.format(states_explored))
for s in optimal_play:
    print(s)

Max score = 0
States explored = 549946
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o | x



In [29]:
# Run this cell to see an optimum game of tic-tac-toe. Note the performance gain WITH alpha-beta pruning.  
blank_game = game_state(tic_tac_toe_board([' ',' ',' ',' ',' ',' ',' ',' ',' ']))
(max_score, states_explored, optimal_play) = score_alpha_beta(blank_game,-2,2)
print('Max score = {}'.format(max_score))
print('States explored = {}'.format(states_explored))
for s in optimal_play:
    print(s)

Max score = 0
States explored = 18297
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o | x



### 1C. <a name="part3"/> </a>Implement Expectimax Search (25 points)

MiniMax and alpha-beta are great, but they both assume that you are playing against an adversary who makes optimal decisions. As anyone who has ever won tic-tac-toe can tell you, this is not always the case. In this question you will implement Expectimax Search, which is useful for modeling probabilistic behavior of agents who may make suboptimal choices. For this version of the problem, you will play against a random adversary.

You will define the following functions:

```python
def maximize_score_expectimax(state)
def expected_score_expectimax(state)
```

Once again, `maximize_score_expectimax` will look very similar to your implementation of `maximize_score`. However, `expected_score_expectimax`'s `optimal_play` should choose amongst successors at random. Assume equal probability of choosing amongst successors. The output format should be the same as the corresponding MiniMax functions.

<div class="alert alert-info">
Implement functions `maximize_score_expectimax(state)` and `expected_score_expectimax(state)` below.
</div>

In [31]:
# Write your code for expectimax search here.

# This is the base case function.
def score_expectimax(state):
    # Initialize states_explored as 1 (this state) and optimal_play (this state).
    states_explored = 1
    optimal_play = [state]
    
    if state.terminal_check():
        terminal_score = state.score()
        return (terminal_score, states_explored, optimal_play)
    elif state.player == 1:
        return maximize_score_expectimax(state)
    elif state.player == 2:
        return expected_score_expectimax(state)

def maximize_score_expectimax(state):
    """Maximizing player (Player 1)."""
    if state.player != 1:
        raise Error

    # Base case: terminal state
    if state.terminal_check():
        return state.score(), 1, [state]

    max_score = -float('inf')
    total_states_explored = 1
    best_play = [state]

    for successor in state.successors():
        succ_score, succ_explored, succ_play = expected_score_expectimax(successor)
        total_states_explored += succ_explored

        if succ_score > max_score:
            max_score = succ_score
            best_play = [state] + succ_play

    return max_score, total_states_explored, best_play


def expected_score_expectimax(state):
    """Chance node (Player 2), which picks randomly (equal probability)."""
    if state.player != 2:
        raise Error

    # Base case: terminal state
    if state.terminal_check():
        return state.score(), 1, [state]

    successors = state.successors()
    if not successors:
        # Defensive: no successors means terminal
        return state.score(), 1, [state]

    total_states_explored = 1
    total_score = 0
    successor_plays = []
    successor_explored_counts = []

    for successor in successors:
        succ_score, succ_explored, succ_play = maximize_score_expectimax(successor)
        total_states_explored += succ_explored
        total_score += succ_score
        successor_plays.append(succ_play)
        successor_explored_counts.append(succ_explored)

    # Expected value: average over all successors
    expected_score = total_score / len(successors)

    # Choose one play path arbitrarily (first one) to represent outcome
    optimal_play = [state] + successor_plays[0]

    return expected_score, total_states_explored, optimal_play


In [32]:
"""Test your expectimax search code here."""
Grader.run_single_test_inline(TestPSet5, "test_3_expectimax", locals())

Test passed!!

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Simulate a Full Game w/ Suboptimal Agent

You're now prepared to run a full game of tic-tac-toe from an initially blank state, this time with an agent that isn't necessarily optimal.

In [33]:
# Run this cell to see a suboptimal game of tic-tac-toe. Run it multiple times. Notice how we always win. 
blank_game = game_state(tic_tac_toe_board([' ',' ',' ',' ',' ',' ',' ',' ',' ']))
(max_score, states_explored, optimal_play) = score_expectimax(blank_game)
print('Max score = {}'.format(max_score))
print('States explored = {}'.format(states_explored))
for s in optimal_play:
    print(s)

Max score = 0.9947916666666667
States explored = 549946
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x | o |  
--+---+--
  |   |  
--+---+--
  |   |  

x | o |  
--+---+--
x |   |  
--+---+--
  |   |  

x | o | o
--+---+--
x |   |  
--+---+--
  |   |  

x | o | o
--+---+--
x | x |  
--+---+--
  |   |  

x | o | o
--+---+--
x | x | o
--+---+--
  |   |  

x | o | o
--+---+--
x | x | o
--+---+--
x |   |  



## <a name="part4"></a> Time Spent on Pset (5 points)

Please use [this form](https://forms.gle/ev8waEiJBN97HxV68) to tell us how long you spent on this pset. After you submit the form, the form will give you a confirmation word. Please enter that confirmation word below to get an extra 5 points. 

In [35]:
form_confirmation_word = "Eomuktang"

In [36]:
# Run all tests
Grader.grade_output([TestPSet5], [locals()], "results.json")
Grader.print_test_results("results.json")

Test passed!!

Test passed!!

Test passed!!

Total score is 105/105.

Score for test_1_minimax (principles_of_autonomy.notebook_tests.pset_5.TestPSet5) is 50/50.

Score for test_2_alpha_beta (principles_of_autonomy.notebook_tests.pset_5.TestPSet5) is 25/25.

Score for test_3_expectimax (principles_of_autonomy.notebook_tests.pset_5.TestPSet5) is 25/25.

Score for test_4_form_word (principles_of_autonomy.notebook_tests.pset_5.TestPSet5) is 5/5.


In [37]:
# Make sure you save the notebook before running this cell so that the most updated version is zipped!
Grader.prepare_submission("ProblemSet05_AdversarialGames_release")

Open-ended responses written to ProblemSet05_AdversarialGames_release_responses_only.ipynb
Compressed files in folder to ../ps5/ps5.zip (submit this to Gradescope)


/usr/local/lib/python3.10/dist-packages/nbformat/__init__.py:132: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
